CLIP (전체 모델) 설명

In [ ]:
class CLIP(nn.Module):
    """
    CLIP 전체 모델

    [구성]
    1. Image Encoder (ResNet or ViT)
    2. Text Encoder (Transformer)
    3. Learnable temperature parameter

    [출처] CLIP 논문: https://arxiv.org/abs/2103.00020
    """
    def __init__(self,
                 embed_dim: int,               # 공통 임베딩 차원 (예: 512)
                 # ===== Vision 관련 =====
                 image_resolution: int,         # 입력 이미지 크기 (예: 224)
                 vision_layers: Union[Tuple[int, int, int, int], int],  # ResNet: tuple, ViT: int
                 vision_width: int,             # Vision encoder의 base width
                 vision_patch_size: int,        # ViT 패치 크기 (ResNet이면 None)
                 # ===== Text 관련 =====
                 context_length: int,           # 최대 토큰 길이 (예: 77)
                 vocab_size: int,               # 어휘 크기 (예: 49408)
                 transformer_width: int,        # Transformer hidden dimension
                 transformer_heads: int,        # Attention head 개수
                 transformer_layers: int        # Transformer layer 개수
                 ):
        super().__init__()

        self.context_length = context_length

        # ========================================
        # Image Encoder 선택: ResNet or ViT
        # ========================================
        if isinstance(vision_layers, (tuple, list)):
            # ===== Option 1: Modified ResNet =====
            # vision_layers가 tuple이면 ResNet
            # 예: (3, 4, 6, 3) -> ResNet-50
            vision_heads = vision_width * 32 // 64  # Attention head 개수 계산
            self.visual = ModifiedResNet(
                layers=vision_layers,          # 각 stage의 블록 개수
                output_dim=embed_dim,          # 최종 출력 차원
                heads=vision_heads,            # AttentionPool의 head 개수
                input_resolution=image_resolution,
                width=vision_width             # Base channel width
            )
        else:
            # ===== Option 2: Vision Transformer =====
            # vision_layers가 int면 ViT
            # 예: 12 -> 12-layer ViT
            vision_heads = vision_width // 64
            self.visual = VisionTransformer(
                input_resolution=image_resolution,  # 224
                patch_size=vision_patch_size,       # 16 or 32
                width=vision_width,                 # Hidden dimension
                layers=vision_layers,               # Transformer layers
                heads=vision_heads,
                output_dim=embed_dim
            )

        # ========================================
        # Text Encoder
        # ========================================
        # Causal attention mask 생성
        # 각 토큰이 자기 자신과 이전 토큰만 볼 수 있게
        self.transformer = Transformer(
            width=transformer_width,           # 예: 512
            layers=transformer_layers,         # 예: 12
            heads=transformer_heads,           # 예: 8
            attn_mask=self.build_attention_mask()  # Causal mask
        )

        self.vocab_size = vocab_size

        # ===== Token Embedding =====
        # 단어 ID -> 임베딩 벡터
        # 예: vocab_size=49408, transformer_width=512
        # -> 49408개 단어 각각을 512차원 벡터로 변환
        self.token_embedding = nn.Embedding(vocab_size, transformer_width)

        # ===== Positional Embedding =====
        # 각 위치(0~76)에 대한 학습 가능한 임베딩
        # Transformer는 순서 정보가 없어서 위치 정보를 명시적으로 추가
        self.positional_embedding = nn.Parameter(
            torch.empty(self.context_length, transformer_width)
        )

        # ===== Final Layer Norm =====
        self.ln_final = LayerNorm(transformer_width)

        # ===== Text Projection =====
        # Transformer 출력(transformer_width)을 이미지와 같은 차원(embed_dim)으로
        self.text_projection = nn.Parameter(torch.empty(transformer_width, embed_dim))

        # ========================================
        # Temperature Parameter (논문의 τ)
        # ========================================
        # Contrastive learning의 핵심 hyperparameter
        # 학습 가능하게 만듦! (고정값이 아님)
        # 초기값: log(1/0.07) ≈ 2.66
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

        # ===== 파라미터 초기화 =====
        self.initialize_parameters()

    def initialize_parameters(self):
        """
        모든 파라미터를 적절한 분포로 초기화

        [중요성]
        초기화가 잘못되면 학습이 불안정하거나 안 될 수 있음
        특히 deep network에서는 초기화가 critical
        """
        # ===== Embedding 초기화 =====
        nn.init.normal_(self.token_embedding.weight, std=0.02)
        nn.init.normal_(self.positional_embedding, std=0.01)

        # ===== Vision Encoder 초기화 =====
        if isinstance(self.visual, ModifiedResNet):
            # AttentionPool 초기화
            if self.visual.attnpool is not None:
                std = self.visual.attnpool.c_proj.in_features ** -0.5
                nn.init.normal_(self.visual.attnpool.q_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.k_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.v_proj.weight, std=std)
                nn.init.normal_(self.visual.attnpool.c_proj.weight, std=std)

            # ResNet block 초기화
            # bn3.weight를 0으로 -> residual path를 처음에는 identity로
            for resnet_block in [self.visual.layer1, self.visual.layer2,
                                self.visual.layer3, self.visual.layer4]:
                for name, param in resnet_block.named_parameters():
                    if name.endswith("bn3.weight"):
                        nn.init.zeros_(param)

        # ===== Text Encoder 초기화 =====
        # Layer가 깊어질수록 작은 std로 초기화 (안정적인 학습)
        proj_std = (self.transformer.width ** -0.5) * ((2 * self.transformer.layers) ** -0.5)
        attn_std = self.transformer.width ** -0.5
        fc_std = (2 * self.transformer.width) ** -0.5

        for block in self.transformer.resblocks:
            nn.init.normal_(block.attn.in_proj_weight, std=attn_std)
            nn.init.normal_(block.attn.out_proj.weight, std=proj_std)
            nn.init.normal_(block.mlp.c_fc.weight, std=fc_std)
            nn.init.normal_(block.mlp.c_proj.weight, std=proj_std)

        # ===== Text Projection 초기화 =====
        if self.text_projection is not None:
            nn.init.normal_(self.text_projection, std=self.transformer.width ** -0.5)

    def build_attention_mask(self):
        """
        Causal attention mask 생성

        [목적]
        각 토큰이 자기 자신과 이전 토큰만 볼 수 있게 제한
        GPT 스타일의 autoregressive attention

        [출력]
        상삼각 행렬 (upper triangular)
        대각선 포함 아래쪽은 0, 위쪽은 -inf

        예시 (context_length=4):
        [[  0, -inf, -inf, -inf],   <- 토큰 0은 자기만 볼 수 있음
         [  0,   0,  -inf, -inf],   <- 토큰 1은 0,1만 볼 수 있음
         [  0,   0,    0,  -inf],   <- 토큰 2는 0,1,2만 볼 수 있음
         [  0,   0,    0,    0 ]]   <- 토큰 3은 모두 볼 수 있음
        """
        mask = torch.empty(self.context_length, self.context_length)
        mask.fill_(float("-inf"))  # 모두 -inf로 채움
        mask.triu_(1)              # 대각선 위쪽만 -inf로 (triu = triangular upper)
        return mask

    @property
    def dtype(self):
        """모델의 dtype (fp16 or fp32)"""
        return self.visual.conv1.weight.dtype

    # ========================================
    # 📷 Image Encoding
    # ========================================
    def encode_image(self, image):
        """
        이미지를 벡터로 인코딩

        [입력] image: (batch, 3, 224, 224)
        [출력] (batch, embed_dim) - 예: (batch, 512)

        [과정]
        이미지 -> Visual Encoder (ResNet/ViT) -> embed_dim 벡터
        """
        return self.visual(image.type(self.dtype))

    # ========================================
    # 📝 Text Encoding
    # ========================================
    def encode_text(self, text):
        """
        텍스트를 벡터로 인코딩

        [입력] text: (batch, context_length) - 토큰 ID 시퀀스
              예: [[49406, 320, 1929, 49407, 0, 0, ...]]
                   [SoT]  [a]  [dog] [EoT] [PAD] ...
        [출력] (batch, embed_dim) - 예: (batch, 512)
        """

        # ===== Step 1: Token Embedding =====
        # 토큰 ID -> 임베딩 벡터
        # (batch, 77) -> (batch, 77, 512)
        x = self.token_embedding(text).type(self.dtype)

        # ===== Step 2: Positional Embedding 추가 =====
        # 각 위치에 대한 정보 더하기
        # (batch, 77, 512) + (77, 512) -> (batch, 77, 512)
        x = x + self.positional_embedding.type(self.dtype)

        # ===== Step 3: Transformer 입력 형태로 변환 =====
        # (batch, 77, 512) -> (77, batch, 512)
        # Transformer는 (sequence, batch, feature) 형태를 기대
        x = x.permute(1, 0, 2)  # NLD -> LND

        # ===== Step 4: Transformer 통과 =====
        # 12개 layer를 거치며 문장의 의미 파악
        # (77, batch, 512) -> (77, batch, 512)
        x = self.transformer(x)

        # ===== Step 5: 다시 원래 형태로 =====
        # (77, batch, 512) -> (batch, 77, 512)
        x = x.permute(1, 0, 2)  # LND -> NLD

        # ===== Step 6: Final Layer Norm =====
        x = self.ln_final(x).type(self.dtype)

        # ===== Step 7: [EoT] 토큰 추출 ⭐ 핵심! =====
        # x.shape = [batch_size, 77, 512]
        #
        # [왜 EoT 토큰을 쓰는가?]
        # Causal attention 때문에 EoT는 모든 이전 토큰을 봤음
        # 따라서 문장 전체 정보가 EoT에 압축되어 있음
        #
        # text.argmax(dim=-1): 각 문장에서 가장 큰 토큰 ID 찾기
        # -> EoT 토큰의 위치 (EoT가 가장 큰 special token ID)
        #
        # 예: text = [49406, 320, 1929, 49407, 0, 0, ...]
        #     argmax = 3 (49407이 가장 큼)
        #     -> x[batch_idx, 3, :] 추출
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.text_projection
        # (batch, 512) @ (512, embed_dim) -> (batch, embed_dim)

        return x

    # ========================================
    # 🔥 Forward Pass (전체 동작)
    # ========================================
    def forward(self, image, text):
        """
        CLIP의 핵심 forward pass

        [입력]
        - image: (batch, 3, 224, 224)
        - text: (batch, 77)

        [출력]
        - logits_per_image: (batch, batch) - 이미지 기준 유사도 행렬
        - logits_per_text: (batch, batch) - 텍스트 기준 유사도 행렬

        [학습 목표]
        대각선 원소가 최대가 되도록 (매칭되는 쌍의 유사도가 높게)
        """

        # ===== Step 1: Encoding =====
        # 이미지와 텍스트를 각각 벡터로 변환
        image_features = self.encode_image(image)    # (batch, embed_dim)
        text_features = self.encode_text(text)       # (batch, embed_dim)

        # ===== Step 2: Normalization ⭐ 매우 중요! =====
        # 벡터를 단위 벡터로 만듦 (길이 = 1)
        #
        # [왜 정규화하는가?]
        # 1. 벡터의 '길이'가 아닌 '방향'만 비교
        # 2. 코사인 유사도 계산 가능
        # 3. 학습 안정성 향상
        #
        # 정규화 안 하면 학습이 불안정해지고 성능 저하!
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)

        # ===== Step 3: Cosine Similarity 계산 =====
        #
        # [Temperature Scaling]
        # logit_scale = exp(τ) 형태
        # 초기값: exp(log(1/0.07)) = 1/0.07 ≈ 14.3
        #
        # [왜 exponential?]
        # τ를 직접 학습하면 음수가 될 수 있음
        # exp를 취하면 항상 양수 보장
        logit_scale = self.logit_scale.exp()

        # [Matrix Multiplication]
        # image_features: (batch, embed_dim)
        # text_features.t(): (embed_dim, batch)
        # 결과: (batch, batch)
        #
        # [의미]
        # logits[i, j] = 이미지 i와 텍스트 j의 유사도 * scale
        logits_per_image = logit_scale * image_features @ text_features.t()
        logits_per_text = logits_per_image.t()  # Transpose

        # ===== Step 4: Return =====
        # 학습 시: 이 logits로 cross entropy loss 계산
        # 추론 시: softmax 취해서 확률로 변환
        return logits_per_image, logits_per_text